In [1]:
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import v_measure_score

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
def ProcessCase(data_path, case_no, output_path = None, rep_count = 20, chain_count = 4):
    rand = []
    mutual_info = []
    vmeasure = []
    ancestral_metric = []
    for rep in range(rep_count):
        rep_path = os.path.join(data_path, "rep" + str(rep))
        case_path = os.path.join(rep_path, "case" + str(case_no))
        truth0 = pd.read_csv(os.path.join(case_path, "cluster_labels.txt"), header=None, names=["ID", "Cluster"])
        ancestral_matrix_truth = np.asarray(pd.read_csv(os.path.join(case_path, "ancestral_matrix.csv"), header=None))
        
        # Check which chain achieved the highest likelihood.
        log_likelihoods = []
        for chain in range(chain_count):
            chain_path = os.path.join(case_path, "genotype", "chain" + str(chain))
            log_likelihood = pd.read_table(os.path.join(chain_path, "joint", "tree0", "log_lik.txt"), header=None, names=["LogLik"])
            log_likelihoods.append(log_likelihood["LogLik"])

        best_chain = np.argmax(log_likelihoods)
        pred_path = os.path.join(case_path, "genotype", "chain" + str(best_chain), "joint", "tree0")
        predicted = pd.read_table(os.path.join(pred_path, "cluster_labels.tsv"), header=None, names=["ID", "Cluster"])
        ancestral_matrix = np.asarray(pd.read_csv(os.path.join(pred_path, "ancestral_matrix.csv"), header=None))

        rand.append(adjusted_rand_score(truth0["Cluster"], predicted["Cluster"]))
        mutual_info.append(adjusted_mutual_info_score(truth0["Cluster"], predicted["Cluster"]))
        vmeasure.append(v_measure_score(truth0["Cluster"], predicted["Cluster"]))
        ancestral_metric.append(np.mean(np.abs(ancestral_matrix_truth - ancestral_matrix)))
        
    df = pd.DataFrame(list(zip(rand, mutual_info, vmeasure, ancestral_metric)), columns=["AdjRandIndex", "AdjMutualInformation", "VMeasure", "AncestralMetric"])
    if output_path is not None:
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        output_file = os.path.join(output_path, "case" + str(case_no) + ".csv")
        df.to_csv(output_file, index=False)
    return(df)


In [4]:
# Write code check that the runs have completed.
depths = ["5"]
dropout_levels = ["90", "95", "98"]
rep_count = 20
case_count = 4
chain_count = 4
state_count = 275
for depth in depths:
    for dropout_level in dropout_levels:
        data_path = "../../data/simul/10X/depth" + depth + "/cn/dropout" + dropout_level + "/"
        finished = True
        for rep in range(rep_count):
            for case in range(case_count):
                for chain in range(chain_count):
                    path_to_final_state = os.path.join(data_path, "rep" + str(rep), "case" + str(case), "genotype", "chain" + str(chain), "states", "tree" + str(state_count))
                    if not os.path.exists(path_to_final_state):
                        print("Depth=%s, Dropout level=%s, rep=%i, case=%i, chain=%i" % (depth, dropout_level, rep, case, chain))
                        finished = False
        if finished:
            print("Depth=%s, Dropout level=%s completed running." % (depth, dropout_level))


Depth=5, Dropout level=90, rep=0, case=0, chain=0
Depth=5, Dropout level=90, rep=0, case=0, chain=1
Depth=5, Dropout level=90, rep=0, case=0, chain=2
Depth=5, Dropout level=90, rep=0, case=0, chain=3
Depth=5, Dropout level=90, rep=0, case=1, chain=0
Depth=5, Dropout level=90, rep=0, case=1, chain=1
Depth=5, Dropout level=90, rep=0, case=1, chain=2
Depth=5, Dropout level=90, rep=0, case=1, chain=3
Depth=5, Dropout level=90, rep=0, case=2, chain=0
Depth=5, Dropout level=90, rep=0, case=2, chain=1
Depth=5, Dropout level=90, rep=0, case=2, chain=2
Depth=5, Dropout level=90, rep=0, case=2, chain=3
Depth=5, Dropout level=90, rep=0, case=3, chain=0
Depth=5, Dropout level=90, rep=0, case=3, chain=1
Depth=5, Dropout level=90, rep=0, case=3, chain=2
Depth=5, Dropout level=90, rep=0, case=3, chain=3
Depth=5, Dropout level=90, rep=1, case=0, chain=0
Depth=5, Dropout level=90, rep=1, case=0, chain=1
Depth=5, Dropout level=90, rep=1, case=0, chain=2
Depth=5, Dropout level=90, rep=1, case=0, chain=3


In [10]:
#depths = ["2", "5"]
depths = ["5"]
dropout_levels = ["90", "95", "98"]
for depth in depths:
    for dropout_level in dropout_levels:
        output_path = "../../_output/simul/depth" + depth + "/wo_cn/dropout" + dropout_level + "/"
        data_path = "../../data/simulation/10X/depth" + depth + "/wo_cn/dropout" + dropout_level + "/"
        df0 = ProcessCase(data_path, 0, output_path)
        df1 = ProcessCase(data_path, 1, output_path)
        df2 = ProcessCase(data_path, 2, output_path)
        df3 = ProcessCase(data_path, 3, output_path)



In [11]:
df3

,AdjRandIndex,AdjMutualInformation,VMeasure,AncestralMetric
0,0.647809,0.800727,0.818799,0.0733
1,0.588479,0.725212,0.751964,0.0981
2,0.546514,0.697742,0.726908,0.1043
3,0.546105,0.684002,0.716783,0.0869
4,0.574897,0.715376,0.743869,0.1280
5,0.683354,0.770614,0.792556,0.1136
6,0.725980,0.821580,0.840863,0.0673
7,0.625907,0.783889,0.801291,0.1000
8,0.535278,0.721325,0.747469,0.0682
9,0.501753,0.726053,0.753748,0.1064


In [ ]:
#depths = ["2", "5"]
depths = ["5"]
dropout_levels = ["90", "95", "98"]
for depth in depths:
    for dropout_level in dropout_levels:
        output_path = "../../_output/simul/depth" + depth + "/cn/dropout" + dropout_level + "/"
        data_path = "../../data/simulation/10X/depth" + depth + "/cn/dropout" + dropout_level + "/"
        df0 = ProcessCase(data_path, 0, output_path)
        df1 = ProcessCase(data_path, 1, output_path)
        df2 = ProcessCase(data_path, 2, output_path)
        df3 = ProcessCase(data_path, 3, output_path)

